In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

LOOKBACK = 7
TEST_DAY = 31

df_7 = pd.DataFrame()

data_file = open('coins.txt', 'r')

lines = data_file.readlines()

coins = [line.strip() for line in lines]

for coin in coins:

    try:
        data = pd.read_csv(f"data/raw_data/{coin}.csv")
    except:
        continue

    tmp_coin_lookback_df = pd.DataFrame()

    for index, row in data.iterrows():

        try:
            
            tmp_df = data.iloc[index:index+LOOKBACK,:]
            tmp_X = tmp_df.iloc[:,4]
            tmp_X = tmp_X.to_numpy().flatten()
            tmp_X = pd.Series(tmp_X)
            tmp_y = data.iloc[index+LOOKBACK,-2]
            tmp_y = pd.Series(tmp_y)
            tmp_var = tmp_X.append(tmp_y, ignore_index=True)
            tmp_var = tmp_var.values.reshape(1, len(tmp_var.values))
            tmp_var = pd.DataFrame(tmp_var)
            tmp_var.columns = [*tmp_var.columns[:-1], 'Close']
            tmp_coin_lookback_df = tmp_coin_lookback_df.append(tmp_var, ignore_index=True)

        except:
            continue

    tmp_coin_lookback_df = tmp_coin_lookback_df.pct_change() * 100
    tmp_coin_lookback_df.dropna(inplace=True)

    if TEST_DAY > tmp_coin_lookback_df.shape[0]:
        TEST_DAY = tmp_coin_lookback_df.shape[0]
    
    tmp_coin_lookback_df.iloc[:-TEST_DAY,:].to_csv(f"data/concat_train/{coin}.csv", index=False)
    tmp_coin_lookback_df.iloc[-TEST_DAY:,:].to_csv(f"data/concat_test/{coin}.csv", index=False)

    df_7 = df_7.append(tmp_coin_lookback_df.iloc[:-TEST_DAY,:])

    print(coin, tmp_coin_lookback_df.shape)

ADA-USD (1651, 8)
ALICE-USD (429, 8)
ALPINE-USD (86, 8)
APE-USD (580, 8)
API3-USD (532, 8)
ARPA-USD (955, 8)
ATOM-USD (1161, 8)
AUDIO-USD (572, 8)
AVAX-USD (605, 8)
AXS-USD (560, 8)
BEL-USD (610, 8)
BNB-USD (1651, 8)
BSW-USD (320, 8)
BTC-USD (2800, 8)
BTT-USD (1203, 8)
BUSD-USD (971, 8)
CHZ-USD (1017, 8)
COCOS-USD (1001, 8)
COS-USD (950, 8)
DAR-USD (195, 8)
DENT-USD (1651, 8)
DOGE-USD (1651, 8)
DOT-USD (547, 8)
ENJ-USD (1651, 8)
ENS-USD (190, 8)
EOS-USD (1651, 8)
ETH-USD (1651, 8)
FIL-USD (1617, 8)
FTM-USD (1296, 8)
GALA-USD (577, 8)
GAL-USD (709, 8)
GMT-USD (400, 8)
GRT-USD (644, 8)
HOT-USD (1579, 8)
ICP-USD (373, 8)
INJ-USD (574, 8)
JASMY-USD (461, 8)
LAZIO-USD (209, 8)
LINK-USD (1651, 8)
LRC-USD (1651, 8)
MANA-USD (1651, 8)
MATIC-USD (1116, 8)
MBOX-USD (405, 8)
MINA-USD (351, 8)
NEAR-USD (581, 8)
NEO-USD (1651, 8)
ONE-USD (1166, 8)
ONT-USD (1532, 8)
PORTO-USD (183, 8)
REEF-USD (505, 8)
ROSE-USD (545, 8)
RVN-USD (1530, 8)
SAND-USD (642, 8)
SANTOS-USD (168, 8)
SHIB-USD (398, 8)
SLP-US

In [2]:
data = df_7
print(data.shape)
data.head()

(66203, 8)


,0,1,2,3,4,5,6,Close
1,-15.393260,1.172610,-12.610706,7.636480,1.635155,0.819670,4.291930,-6.551851
2,1.172610,-12.610706,7.636480,1.635155,0.819670,4.291930,-6.551851,2.312497
3,-12.610706,7.636480,1.635155,0.819670,4.291930,-6.551851,2.312497,6.503852
4,7.636480,1.635155,0.819670,4.291930,-6.551851,2.312497,6.503852,5.330432
5,1.635155,0.819670,4.291930,-6.551851,2.312497,6.503852,5.330432,-4.229063


In [3]:
from sklearn.model_selection import train_test_split

data = data.reset_index(drop=True)

X = data.iloc[:,:-1].values
y = data.iloc[:,-1]


print(X.shape)
print(y.shape)

(66203, 7)
(66203,)


In [4]:
import numpy as np

X = X.astype(np.float32)
y = y.astype(np.float32)

X = np.nan_to_num(X.astype(np.float32))
y = np.nan_to_num(y.astype(np.float32))

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import numpy as np

rf = RandomForestRegressor(n_estimators=200, max_depth=20)

rf.fit(X, y)

preds = rf.predict(X)

mape = mean_absolute_percentage_error(preds, y)
rmse = mean_squared_error(preds, y, squared=False)

print(mape, rmse)
print()

104.1377843300831 174.93373825803025



In [6]:
import warnings
warnings.filterwarnings('ignore')

mape_arr = {}
rmse_arr = {}

for coin in coins:
    print("Coin: ", coin)

    try:
        test_data = pd.read_csv(f"data/concat_test/{coin}.csv")
    except:
        continue

    test_X = test_data.iloc[:,:-1]
    test_y = test_data.iloc[:,-1]

    preds = rf.predict(test_X)

    np.savetxt(f"data/pred/{coin}.csv", preds, delimiter=",")
        
    mape = mean_absolute_percentage_error(preds, test_y)
    rmse = mean_squared_error(preds, test_y, squared=False)

    mape_arr[coin] = mape
    rmse_arr[coin] = rmse

    print("MAPE: ", mape * 100)
    print("RMSE: ", rmse)
    print()

Coin:  ADA-USD
MAPE:  3185.798819625251
RMSE:  7.821227938699729

Coin:  ALICE-USD
MAPE:  3232.461385014406
RMSE:  11.711897250162789

Coin:  ALPINE-USD
MAPE:  13202.759439194848
RMSE:  25.007991663180626

Coin:  APE-USD
MAPE:  1985.7978310635556
RMSE:  7.576701980295129

Coin:  API3-USD
MAPE:  4025.0167580510356
RMSE:  11.73053663039685

Coin:  ARPA-USD
MAPE:  2381.6276885762436
RMSE:  10.36348065283174

Coin:  ATOM-USD
MAPE:  3156.6763757213334
RMSE:  8.258443551349078

Coin:  AUDIO-USD
MAPE:  8139.8523997907005
RMSE:  11.045037795973014

Coin:  AVAX-USD
MAPE:  4098.527645808935
RMSE:  9.204205128227699

Coin:  AXS-USD
MAPE:  3100.994034959479
RMSE:  8.459446183258207

Coin:  BEL-USD
MAPE:  3270.451542245106
RMSE:  13.011725450143162

Coin:  BNB-USD
MAPE:  16201.131535806639
RMSE:  5.359895745161623

Coin:  BSW-USD
MAPE:  3685.595693610268
RMSE:  9.848137568172943

Coin:  BTC-USD
MAPE:  4133.221166070846
RMSE:  4.04385201473522

Coin:  BTT-USD
MAPE:  5002.739653746103
RMSE:  5.929539

In [12]:
def day_end_money_calculation(arr, top_N, money, mape):
    
    new_arr = arr[0:top_N]
    
    print(new_arr)
    # weights = [i[1] / mape[i[0]] for i in new_arr]
    weights = [i[1] / 1 for i in new_arr]

    sum_weights = sum(weights)

    normalized_weights = [i/sum_weights for i in weights]

    change_ratio = [i[-1] for i in new_arr]
    
    cum_return = [normalized_weights[i] * money * ((change_ratio[i]+100)/100) for i in range(top_N)]

    cum_return = sum(cum_return)

    return cum_return

In [13]:
money_1 = 100

top_N = 7

islem = 0

for day in range(TEST_DAY):

    values = []

    for coin in coins:

        try:
                
            preds = pd.read_csv(f"data/pred/{coin}.csv", header=None)
            truths = pd.read_csv(f"data/concat_test/{coin}.csv", header=None)

            today_pred = float(preds.iloc[day,0])
            # yesterday_pred = float(preds.iloc[day-1,0])

            today_real = float(truths.iloc[day+1,-1])
            # yesterday_real = float(truths.iloc[day,-1])
            
            metric_1 = today_pred
            metric_2 = today_real

            ratio = today_real

            mape = rmse_arr[coin]

            values.append((coin, metric_1, metric_2, ratio))

        except:

            continue
        
    metric_1_values = sorted(values, key=lambda x: x[1], reverse=True)
    # print(metric_1_values)
    money_1 = day_end_money_calculation(metric_1_values, top_N, money_1, rmse_arr)
    print(day, money_1)
    print()


[('PORTO-USD', 0.8928159068480945, -8.76720187748673, -8.76720187748673), ('TRX-USD', 0.7940309850711169, -0.557106602151014, -0.557106602151014), ('ALPINE-USD', 0.6950967600594395, -4.590212600673748, -4.590212600673748), ('GAL-USD', 0.6291334310719803, -2.3102081902029004, -2.3102081902029004), ('ONT-USD', 0.5556119571770961, -0.4088894925073805, -0.4088894925073805), ('ARPA-USD', 0.5432021714848064, -4.932583600263907, -4.932583600263907), ('JASMY-USD', 0.44639862028218646, -7.667840218341415, -7.667840218341415)]
0 95.7765163481157

[('PORTO-USD', 1.3157078179196606, -9.636952038232527, -9.636952038232527), ('APE-USD', 0.9989939608546656, -8.344866150119024, -8.344866150119024), ('COS-USD', 0.9093887843148658, -7.496277269798501, -7.496277269798501), ('GRT-USD', 0.9087220318119464, -2.3753251198792347, -2.3753251198792347), ('DOT-USD', 0.9050442605124989, -23.495304979529163, -23.495304979529163), ('CHZ-USD', 0.8941721975466825, -1.4802792443210144, -1.4802792443210144), ('JASMY-US

In [9]:
# Average train MAPE - in this case RMSE is calculated by % regression so it is same as standard MAPE
sum(rmse_arr.values()) / len(rmse_arr.values())

9.832330730499816

In [10]:
rf.predict([[-10.248898118507721,11.263091152021376,3.1442183437854654,4.570659504823826,-8.313838278043828,9.54847460387791,-4.908319014517359]])

array([0.34325613])

In [11]:
# array([-0.27237273])